In [1]:
#############################################################################
####################Import all packages######################################
#############################################################################
import os
import random
import numpy as np
#import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
#import pandas_profiling as pp
import pandas as pd
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.mode.chained_assignment = None  # default='warn'
# Set working directory
#############################################################################
# Make Jupyter Notebook show ALL output of a cell, not only the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
def read_all_csvs_in_folder(path):
    ''' Function reading in all CSV files in a given folder.
        1. First lists all files in folder.
        2. Then reads in only files which are NOT of file size = 0 and contain headers + AT LEAST 1 row of data
        3. Appends all files into a final dataframe
        4. Also prints the amount of files in the folder, and the amount of files used for the final dataframe.
    '''
    import pandas as pd
    import glob
    import os
    # Create list containing all files names of the current folder
    all_files = glob.glob(path + "/*.csv")
    # These empty lists will be filled with:
    # 1. the names of the CSV files we want to append/merge, 
    # 2. 2 lists containing numbers representing all the files that were empty (or headers but no row data) and not read
    
    list_with_dfs = []
    zero_files = []
    empty_files = []
    empty_row_files = []
    filename_list = []
    n_files_appended = 0
    
    print('')
    print('')
    print('#####################################################')
    print('STARTING READ IN')
    print('#####################################################')
    print('')
    
    for filename in all_files:        
        print('Reading current file:')
        print(filename)
        # Check whether the CSV file is larger than 0Bytes (if not, it has for sure no data and will break during read in)
        try:        
            if os.path.getsize(filename) == 0:
                zero_files.append(filename)                
                print('')
                print('------------------------------------------------------')
                print('!!! SKIPPING FILE: ' + filename)
                print('    REASON: FILE SIZE == 0')
                print('------------------------------------------------------')
                print('')
            if os.path.getsize(filename) > 0:
                df = pd.read_csv(filename, index_col=None, header=0)            
                if df.empty:
                    empty_row_files.append(filename)
                    print('')
                    print('------------------------------------------------------')
                    print('!!! SKIPPING FILE: ' + filename)
                    print('    REASON: FILE CONTAINS NO ROW DATA')
                    print('------------------------------------------------------')
                    print('')
                # Check whether the CSV is empty (this ALSO works when it DOES have headers, but no row data. This is important!)
                if not df.empty:
                    list_with_dfs.append(df)
                    n_files_appended = n_files_appended + 1
                    filename_list.append(filename)
        except pd.errors.EmptyDataError:
            print('')
            print('------------------------------------------------------')
            print('!!! SKIPPING FILE: ' + filename)
            print('    REASON: FILE ENTIRELY EMPTY WITHOUT HEADERS')
            print('------------------------------------------------------')
            print('')
            empty_files.append(filename)            
            pass
    
    # Merge all files in the file list
    frame = pd.concat(list_with_dfs, axis=0, ignore_index=True)
    print('')
    print('#####################################################')
    print('!!! DONE READING & MERGING DATA !!!')
    print('#####################################################')
    print('')
    print('------------------------------------------------------')
    print('All files: ' + str(len(all_files)))    
    print('')
    print('Successfully appended files: ' + str(n_files_appended))
    print('------------------------------------------------------')
    print('------------------------------------------------------')
    print('')
    print('Empty files without headers: ' + str(len(empty_files)))
    #print(len(empty_files))
    print(empty_files)
    print('')
    print('Zero size files: ' + str(len(zero_files)))
    #print(len(zero_files))
    print(zero_files)
    print('')
    print('Empty row files: ' + str(len(empty_row_files)))
    #print(len(empty_row_files))
    print(empty_row_files)
    print('')    
    print('#####################################################')
    
    return frame    

In [6]:
# Read in files and merge them

#PreFlight & Flight
#path_pre_and_flight = r'../data/experiment/raw/exp_raw_pre_and_flight/' 
#pre_and_flight = read_all_csvs_in_folder(path_pre_and_flight)

path_postflight1 = '/home/taco/Documents/greta_forarex/new_data/postflight_control_FINAL/Postflightdata_1/'
#path_postflight2 = '/home/taco/Documents/greta_forarex/Neue gesammelten Daten für Timmy_08082019/Flight Experiment_Postflight_raw_20190808/Postflightdata von 15032019 bis 11042019'

# PostFlight
postflight_1 = read_all_csvs_in_folder(path_postflight1)
#postflight_2 = read_all_csvs_in_folder(path_postflight2)

# Extra Flight
#path_extra = r'../data/experiment/raw/exp_raw_extra/2019-04-12(1)/2019-04-12/' 
#extra = read_all_csvs_in_folder(path_extra)




In [8]:
## Remove all unncessary columns
cols_to_keep = ['timeStamp',
                'Exp0_OxygenTemp', 'Exp0_OxygenpercentO2', 'Exp0_PhValue',
                'Exp1_OxygenTemp', 'Exp1_OxygenpercentO2',
                'Pressure_LateAccess']

#pre_and_flight = pre_and_flight[cols_to_keep]
postflight_1 = postflight_1[cols_to_keep]
#extra = extra[cols_to_keep]

In [25]:
# DFs contain negligible amount of missing values (NAs), just drop them
#import missingno as msno
#msno.matrix(postflight_1)
#pre_and_flight.dropna(inplace=True)
postflight_1.dropna(inplace=True)
#extra.dropna(inplace=True)

In [14]:
# Naive DateTime TimeStamp needs to be increased by 1hour (UTC+1 for Sweden) compared to what was recorded (apparently UTC, UK)
#pre_and_flight.loc[:,'timeStamp'] = pd.to_datetime(pre_and_flight.timeStamp, unit='s')
postflight_1.loc[:,'timeStamp'] = pd.to_datetime(postflight_1.timeStamp, unit='s')
#extra.loc[:,'timeStamp'] = pd.to_datetime(extra.timeStamp, unit='s').dt.tz_localize('UTC').dt.tz_convert('Europe/Stockholm')


In [22]:
# Check the data range of the final timestamps
postflight_1.timeStamp.min()
postflight_1.timeStamp.max()



Timestamp('2019-03-11 11:52:55')

Timestamp('2019-03-14 18:53:49')

In [17]:
# Calculate difference between time when it was actually recorded and incorrect system time stamp
time_delta = pd.to_datetime('2019-03-11 13:20:00') - pd.to_datetime('2000-01-01 14:30:02') 

In [19]:
# add delta to timestamp
postflight_1.timeStamp = postflight_1.timeStamp + time_delta


In [21]:
postflight_1.sort_values('timeStamp', inplace=True)

In [29]:
postflight_1.head(10)


,timeStamp,Exp0_OxygenTemp,Exp0_OxygenpercentO2,Exp0_PhValue,Exp1_OxygenTemp,Exp1_OxygenpercentO2,Pressure_LateAccess,year,month,day,hour,minute,second
15600,2019-03-11 11:52:55,0.000,0.000,0.000,0.000,0.000,716.1,2019,3,11,11,52,55
15601,2019-03-11 11:53:01,14.319,9.880,0.000,12.475,5.796,716.1,2019,3,11,11,53,1
15602,2019-03-11 11:53:05,14.314,8.315,6.924,12.477,4.949,716.3,2019,3,11,11,53,5
15603,2019-03-11 11:53:11,14.314,8.334,6.924,12.485,4.940,716.6,2019,3,11,11,53,11
15604,2019-03-11 11:53:17,14.316,8.343,6.925,12.485,4.932,716.6,2019,3,11,11,53,17
15605,2019-03-11 11:53:23,14.316,8.347,6.924,12.490,4.945,717.0,2019,3,11,11,53,23
15606,2019-03-11 11:53:28,14.316,8.343,6.925,12.493,4.938,717.4,2019,3,11,11,53,28
15607,2019-03-11 11:53:35,14.309,8.352,6.925,12.498,4.945,717.9,2019,3,11,11,53,35
15608,2019-03-11 11:53:41,14.316,8.323,6.926,12.503,4.934,717.8,2019,3,11,11,53,41
15609,2019-03-11 11:53:46,14.311,8.343,6.926,12.503,4.924,718.1,2019,3,11,11,53,46


In [25]:
# Create different time epochs for convenient plotting & exploration
# Postflight
postflight_1.loc[:, 'year'] = postflight_1.timeStamp.dt.year
postflight_1.loc[:, 'month'] = postflight_1.timeStamp.dt.month
postflight_1.loc[:, 'day'] = postflight_1.timeStamp.dt.day
postflight_1.loc[:, 'hour'] = postflight_1.timeStamp.dt.hour
postflight_1.loc[:, 'minute'] = postflight_1.timeStamp.dt.minute
postflight_1.loc[:, 'second'] = postflight_1.timeStamp.dt.second


In [27]:
# Scale columns values 
# Postflight
# All divided by 10000
postflight_1.loc[:,'Exp0_OxygenTemp'] = postflight_1.Exp0_OxygenTemp/1000
postflight_1.loc[:,'Exp1_OxygenTemp'] = postflight_1.Exp1_OxygenTemp/1000
postflight_1.loc[:,'Exp0_OxygenpercentO2'] = postflight_1.Exp0_OxygenpercentO2/1000
postflight_1.loc[:,'Exp1_OxygenpercentO2'] = postflight_1.Exp1_OxygenpercentO2/1000
postflight_1.loc[:,'Exp0_PhValue'] = postflight_1.Exp0_PhValue/1000
# Pressure divided by 10
postflight_1.loc[:, 'Pressure_LateAccess'] = postflight_1.Pressure_LateAccess/10



In [30]:
# Saving to disk the merged files (not yet processed)
# In Pickle format for fast read in
#pre_and_flight.to_pickle('../data/experiment/merged/pre_and_flight_merged.pkl')
postflight_1.to_pickle('../new_data/cleaned/postflight_1_clean.pkl')
#extra.to_pickle('../data/experiment/merged/extra_merged.pkl')
# In CSV
#pre_and_flight.to_csv('../data/experiment/merged/pre_and_flight_merged.csv')
postflight_1.to_csv('../new_data/cleaned/postflight_1_clean.csv')
#extra.to_csv('../data/experiment/merged/extra_merged.csv')
